Karla Jacobo
DSCI 8950
EDA Assignment

To start off the Exploratory Data Analysis, I am going to import the data from my team's selected data sources and store them in local variables for easier access. 

In [138]:
import numpy as np
import pandas as pd

airline = pd.read_csv('../data/openFlightsRaw/airline.csv')
airports = pd.read_csv('../data/openFlightsRaw/airports.csv')
countries = pd.read_csv('../data/openFlightsRaw/countries.csv')
planes = pd.read_csv('../data/openFlightsRaw/planes.csv')
routes = pd.read_csv('../data/openFlightsRaw/routes.csv')

In the next block of code, I am going to gather the following information from the datasets:
    - Number of Rows
    - Number of Columns
    - Data types

In [183]:
tables = [airline, airports, countries, planes, routes]
allTableTypes = []
airline.name = 'Airline'
airports.name = 'Airports'
countries.name = 'Countries'
planes.name = 'Planes'
routes.name = 'Routes'

for table in tables:
    print(table.name + ' table size: ' + str(table.shape[0]) + ' rows, ' + str(table.shape[1]) + ' columns')

print()

for table in tables:
    dataTypes = table.dtypes.values

    for type in dataTypes:
        allTableTypes.append(str(np.dtype(type)))

print('All data types featured in tables: ')
print(set(allTableTypes))


Airline table size: 6161 rows, 8 columns
Airports table size: 7697 rows, 14 columns
Countries table size: 260 rows, 3 columns
Planes table size: 245 rows, 3 columns
Routes table size: 67662 rows, 9 columns

All data types featured in tables: 
{'int64', 'object', 'float64'}
